# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
from sklearn.cluster import (
    DBSCAN,
    OPTICS,
    AffinityPropagation,
    AgglomerativeClustering,
    KMeans,
    MeanShift,
)
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
composantes_acp = pd.read_csv("data/composantes_acp.csv", index_col="ID")
composantes_acm = pd.read_csv("data/composantes_acm.csv", index_col="ID")

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

var_categoriques_extra = ["NbAcceptedCampaigns", "HasAcceptedCampaigns", "NbChildren"]

var_categoriques_fe = var_categoriques + var_categoriques_extra

# Clustering

## Préparation des données

Nous commencer par fusionner les variables quantitatives et les coordonnées des individus dans l'ACM.

In [ ]:
X_clust = pd.concat((df[var_numeriques], composantes_acm), axis=1)

In [ ]:
X_clust.head()

In [ ]:
scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(X_clust),
    columns=X_clust.columns,
    index=df.index,
)

In [ ]:
df_apres_scale.head()

In [ ]:
df_avec_clusters = df_apres_scale.copy()

## Différents algorithmes de clusters

Nous choisissons de tester 2 types de modèles de clustering :
1) les modèles à choix du nombre de clusters (KMeans, GMM, CAH)
2) les modèles qui décident du nombre de clusters (OPTICS, MeanShift et AffinityPropagation)

Cela nous permettra de comparer le nombre de clusters donné par les seconds algorithmes.

Pour les modèles pour lesquels il faut choisir le nombre de clusters, nous décidons de tester des clusters de taille 2 à 5 (inclus), car un trop grand nombre de clusters serait plus difficile à interpréter pour l'équipe marketing dans un premier temps.

In [ ]:
NB_CLUSTER_MIN = 2
NB_CLUSTER_MAX = 6  # non inclus

In [ ]:
dict_kmeans = {
    f"KMeans{i}": KMeans(n_clusters=i, random_state=SEED)
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_gmm = {
    f"GMM{i}": GaussianMixture(
        n_components=i, covariance_type="full", random_state=SEED
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_ward = {
    f"CAH (Ward) {i}": AgglomerativeClustering(n_clusters=i)
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_average = {
    f"CAH (average linkage) {i}": AgglomerativeClustering(
        n_clusters=i, linkage="average"
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_simple = {
    f"CAH (single linkage) {i}": AgglomerativeClustering(n_clusters=i, linkage="single")
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_complete = {
    f"CAH (complete linkage) {i}": AgglomerativeClustering(
        n_clusters=i, linkage="complete"
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

In [ ]:
model_clusters = {
    **dict_kmeans,
    **dict_gmm,
    **dict_cah_ward,
    **dict_cah_average,
    **dict_cah_simple,
    **dict_cah_complete,
    "OPTICS": OPTICS(),
    "MeanShift": MeanShift(),
    "AffinityPropagation": AffinityPropagation(random_state=SEED),
}

In [ ]:
cluster_metrics = []

for (model_name, model) in model_clusters.items():
    if model.__class__ == GaussianMixture:  # cas particulier du mélange gaussien
        df_avec_clusters[model_name] = model.fit_predict(df_apres_scale)
    else:
        model.fit(df_apres_scale)
        df_avec_clusters[model_name] = model.labels_

    df_avec_clusters[model_name] = pd.Categorical(
        df_avec_clusters[model_name].astype(str)
    )

    nb_clusters = df_avec_clusters[model_name].nunique()

    repartition = list(
        df_avec_clusters[model_name].value_counts(normalize=True).round(2).astype(str)
    )  # todo: enlever astype(str) si ça sert à rien (tester)

    cluster_metrics.append(
        [
            model_name,
            nb_clusters,
            " | ".join(repartition),
            silhouette_score(
                df_apres_scale, df_avec_clusters[model_name]
            ),  # proche de 1 = mieux
            calinski_harabasz_score(
                df_apres_scale,
                df_avec_clusters[model_name],
            ),  # plus élevé, mieux c'est
            davies_bouldin_score(
                df_apres_scale, df_avec_clusters[model_name]
            ),  # proche de 0 = mieux
        ]
    )

In [ ]:
pd.DataFrame(
    cluster_metrics,
    columns=[
        "Algorithme de clustering",
        "Nombre de clusters",
        "Répartition",
        "Silhouette",
        "Calinski-Harabasz",
        "Davies-Bouldin",
    ],
)

In [ ]:
# todo: tableau pour dire quels clusters on choisit d'étudier

## Visualisation

In [ ]:
def affiche_taille_clusters(nom_cluster):
    plt.title("Taille des clusters")
    sns.histplot(df_avec_clusters[nom_cluster], shrink=0.5)

    plt.show()

In [ ]:
def affiche_clusters_acp(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACP 1-2")
    ax[1].set_title("Clusters sur les axes d'ACP 3-4")

    sns.scatterplot(
        composantes_acp,
        x="ACP1",
        y="ACP2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )
    sns.scatterplot(
        composantes_acp,
        x="ACP3",
        y="ACP4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_acm(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACM 1-2")
    ax[1].set_title("Clusters sur les axes d'ACM 3-4")

    sns.scatterplot(
        composantes_acm,
        x="ACM1",
        y="ACM2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )

    sns.scatterplot(
        composantes_acm,
        x="ACM3",
        y="ACM4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_var_quanti(nom_cluster):
    """Affiche les variables quantitatives en fonction des clusters."""
    for var in var_numeriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 3))

        sns.boxplot(
            x=df[var],
            y=df_avec_clusters[nom_cluster],
            width=0.25,
            ax=ax[0],
        )

        sns.histplot(
            x=df[var],
            kde=True,
            ax=ax[1],
            hue=df_avec_clusters[nom_cluster],
            stat="probability",
            common_norm=False,
        )

        plt.show()

In [ ]:
def affiche_clusters_var_quali(nom_cluster):
    """Affiche les variables qualitatives en fonction des clusters et vice-versa."""
    for var in var_categoriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))

        sns.histplot(
            x=df[var].astype(str),
            ax=ax[0],
            hue=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        sns.histplot(
            hue=df[var].astype(str),
            ax=ax[1],
            x=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        plt.show()

In [ ]:
def affiche_clusters(nom_cluster):
    """Affiche les variables en fonction des clusters."""
    affiche_taille_clusters(nom_cluster)
    affiche_clusters_acp(nom_cluster)
    affiche_clusters_acm(nom_cluster)

    affiche_clusters_var_quanti(nom_cluster)
    affiche_clusters_var_quali(nom_cluster)

In [ ]:
affiche_clusters("KMeans3")

In [ ]:
affiche_clusters("KMeans3")

In [ ]:
affiche_clusters("CAH (Ward) 3")

In [ ]:
# todo: faire une conclusion sur les 3 types de client-type (décrire les clients types)

# Sauvegarde du Dataframe

In [ ]:
# cluster3 = pd.DataFrame(k3.labels_, columns=["cluster3"])

In [ ]:
# cluster3.to_csv("data/clusters-kmeans3.csv")